# AOC 2020

## Day 3
### Part 1

In [ ]:
with open('input03.txt') as fin:
    lines = fin.readlines()
    trees = [line.strip() for line in lines]
    height, width = len(trees), len(trees[0])

In [ ]:
def step(x, y, dx, dy):
    return x+dx, y+dy

def run(dx=3, dy=1):
    cx, cy = 0, 0
    nb_trees = 0
    while cy < height:
        if trees[cy][cx] == '#':
            nb_trees += 1
        cx, cy = step(cx, cy, dx, dy)
        cx = cx % width
    return nb_trees

In [ ]:
run()

### Part 2

In [ ]:
slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
count = 1
for dx, dy in slopes:
    count *= run(dx, dy)
print(count)

## Day 2
### Part 1

In [ ]:
import re
from collections import Counter

def parse(line):
    low, high, letter, password = re.split(r"[- ]", line.strip())
    
    low, high, letter = int(low), int(high), letter[:-1]
    
    return low, high, letter, password

def valid(line):
    low, high, letter, password = parse(line)
    count = Counter(password)
    
    return count[letter] >= low and count[letter] <= high

with open('input02.txt') as fin:
    lines = fin.readlines()
    
    print(len([line for line in lines if valid(line)])) 

### Part 2

In [ ]:
def valid_p2(line):
    low, high, letter, password = parse(line)
    
    return (password[low - 1] == letter) != (password[high - 1] == letter)

In [ ]:
with open('input02.txt') as fin:
    lines = fin.readlines()
    
    print(len([line for line in lines if valid_p2(line)])) 

## Day 1
### Part 1

In [ ]:
with open('input01.txt') as fin:
    data = [int(d.strip()) for d in fin.readlines()]

In [ ]:
for a in data:
    for b in data:
        if a+b == 2020:
            print(a*b)

### Part 2

In [ ]:
for a in data:
    for b in data:
        for c in data:
            if a+b+c == 2020:
                print(a*b*c)